In [9]:
import os
import re
import pandas as pd
import MeCab
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
from future.utils import iteritems
from collections import Counter
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from ckonlpy.tag import Twitter
from collections import Counter
# from konlpy.tag import Okt

In [2]:
def preprocess(path, idx):
   
   # 폴더 내 모든 파일 부르기
    files_group = []    
    files_group = [ word for word in os.listdir(path) if word[-3:] == 'csv']
    
    # i 번째 파일 식당 부르기
    i = idx
    print('총', len(files_group), '개 중 ',i + 1, '번 째 파일입니다.' )
    df1 = pd.read_csv(path + files_group[i])
    
    # title 문장 
    sents = []
    for i in range(len(df1)):
        sents.append(re.sub('[^가-힣a-zA-Z\s\d]' ,'', df1.title.iloc[i]))
        
    # 본문 문장 합치기
    for i in range(len(df1)):
        try:
            sents.append(re.sub('[^가-힣a-zA-Z\s\d]' ,'', df1.description.iloc[i]))
        except:
            pass
    
    print('총 %s개의 문장이 추출되었습니다.' % len(sents))
    
    
    ## 전처리
    m = MeCab.Tagger()
    mecab_sents = [m.parse(sent) for sent in sents]
    
    add_nouns= []
    except_words = []
    
    with open('./data/restaurants/food_dic.txt', 'r', encoding = 'utf-8') as f:
        s = f.readlines()
    foods = [word.replace('\n', '') for word in s]
    
    for ddd in mecab_sents:
        lines = ddd.split('\n')
        tmp1 = [line.split('\t')[0] for line in lines if 'Compound' in line ]
        tmp2 = [line.split('\t')[0] for line in lines if ('MAG' in line or 'EF' in line) and len(line.split('\t')[0]) > 1]
        add_nouns.extend(tmp1)
        except_words.extend(tmp2)
    add_nouns.extend(foods)
    
    add_nouns = list(set(add_nouns))
    except_words = list(set(except_words))
    
    
    m = Twitter()
    # 추가할 사용자 사전
                 # 식당 이름 
    user_dic = [df1.name.iloc[0], '기대감', '웨이팅', '가성비', '재방문', '오리지널', '오리지날',  
                '콜키지', '가정식', '인스타', '콜키지프리', '콜키지무료', '회식', '갬성', 'sns', 'SNS','회식', 
                '상견례', '연말모임', '기념일', '생일', '생신', '소개팅',
                # 누구랑
                '남친', '남자친구', '여친', '여자친구', '부모님', '어른', '회사동기','동기','친구',
                # 방송 tv
                '수요미식회', '3대천왕', '맛있는녀석들', '먹거리파일', '백종원', '골목식당', '착한식당', '르꼬르동블루', 
                '블루리본서베이', '미슐랭', '티비', '방송',
               '서베이', '망고플레이트', '망플', '빕구르망',
                # 그외
               '모던한', '퓨전', '현대적','핫플레이스', '경리단길', '할로윈'
               ]
    user_dic.extend(add_nouns)
    
    # 제외할 사용자 사전
    except_dic = ['어요','오랜만', '입니당', '먹음', '입니다', '안녕', '이번', '정말', '부터', '오늘', '어서', '있는', '래서', '드디어', '요즘', '다가', '한번'
               , '위치', '홍대', '근처', '바로', '좋은', '후기', '가격', '메뉴', '입구', '서울', '항상', '찾기', '메뉴판', '아하', '여기', '진짜', '보고', '면서',
               '좋고', '좋아', '까지', '려고', '있다', '여기', '포스팅', '하기', '추천', '맛집', '정도', '시간', '가게', '가득', '전부']
    except_dic.append(except_words)


    # Mecab으로 추가할 단어 추가
    for word in user_dic:
        m.add_dictionary(word, 'Noun')

    # Mecab으로 제외할 단어 추가
    for word in except_dic:
        m.add_dictionary(word, 'Josa')
    mecab_sents = [m.pos(sent) for sent in sents]

    # pos 태깅하여 명사만 걸러냄
    mecab_sents2 = []
    for output in mecab_sents:
        tmp = []
        for token in output:
            if (len(token[0]) > 1 and token[0] != '빵') and (token[1] == 'Noun'  ) and (token[0] not in except_dic):
                    tmp.append(token[0])
        mecab_sents2.append(' '.join(tmp))
    
    # 식당 이름
    name = df1.name.iloc[0]
    
    # 네이버 태그 리스트
    tags = df1['tags'].iloc[0]
    
    return mecab_sents2, name, mecab_sents, except_dic, tags 

In [3]:
## textrank

def scan_vocabulary(sents, tokenize, min_count=2):
    counter = Counter(w for sent in sents for w in tokenize(sent) if len(w) > 1 )
    counter = {w:c for w,c in counter.items() if c >= min_count}
    idx_to_vocab = [w for w, _ in sorted(counter.items(), key=lambda x:-x[1])]
    vocab_to_idx = {vocab:idx for idx, vocab in enumerate(idx_to_vocab)}
    return idx_to_vocab, vocab_to_idx

from collections import defaultdict

def cooccurrence(tokens, vocab_to_idx, window=2, min_cooccurrence=2):
    counter = defaultdict(int)
    for s, tokens_i in enumerate(tokens):
        vocabs = [vocab_to_idx[w] for w in tokens_i if w in vocab_to_idx]
        n = len(vocabs)
        for i, v in enumerate(vocabs):
            if window <= 0:
                b, e = 0, n
            else:
                b = max(0, i - window)
                e = min(i + window, n)
            for j in range(b, e):
                if i == j:
                    continue
                counter[(v, vocabs[j])] += 1
                counter[(vocabs[j], v)] += 1
    counter = {k:v for k,v in counter.items() if v >= min_cooccurrence}
    n_vocabs = len(vocab_to_idx)
    return dict_to_mat(counter, n_vocabs, n_vocabs)

from scipy.sparse import csr_matrix

def dict_to_mat(d, n_rows, n_cols):
    rows, cols, data = [], [], []
    for (i, j), v in d.items():
        rows.append(i)
        cols.append(j)
        data.append(v)
    return csr_matrix((data, (rows, cols)), shape=(n_rows, n_cols))

def word_graph(sents, tokenize=None, min_count=2, window=2, min_cooccurrence=2):
    idx_to_vocab, vocab_to_idx = scan_vocabulary(sents, tokenize, min_count)
    tokens = [tokenize(sent) for sent in sents]
    g = cooccurrence(tokens, vocab_to_idx, window, min_cooccurrence)
    return g, idx_to_vocab

import numpy as np
from sklearn.preprocessing import normalize

def pagerank(x, df=0.85, max_iter=30):
    assert 0 < df < 1

    # initialize
    A = normalize(x, axis=0, norm='l1')
    R = np.ones(A.shape[0]).reshape(-1,1)
    bias = (1 - df) * np.ones(A.shape[0]).reshape(-1,1)

    # iteration
    for _ in range(max_iter):
        R = df * (A * R) + bias

    return R

def textrank_keyword(sents, tokenize, min_count, window, min_cooccurrence, df=0.85, max_iter=30, topk=30):
    g, idx_to_vocab = word_graph(sents, tokenize, min_count, window, min_cooccurrence)
    R = pagerank(g, df, max_iter).reshape(-1)
    idxs = R.argsort()[-topk:]
    keywords = [(idx_to_vocab[idx], R[idx]) for idx in reversed(idxs)]
    return keywords

In [4]:
from soynlp.tokenizer import MaxScoreTokenizer

def textrank(mecab_sents2, name):
    scores = {'이태원': 0.3, '한강진': 0.3, '경리단길': 0.3, '신촌': 0.3, '홍대': 0.3, name: 0.7, '홍대입구': 0.2, '맛집':0.1, '추천': 0.1, '남자친구': 0.5, '친구': 0.5, '부모님': 0.5, '언니': 0.5, '동기': 0.5}

    tokenizer = MaxScoreTokenizer(scores=scores)

    textrank_keywords = textrank_keyword(mecab_sents2, tokenizer, len(mecab_sents2) // 100 + 1, 5, len(mecab_sents2) // 100 + 1)
    return textrank_keywords

In [5]:
## word2vec * tfidf

def word2vec_tfidf(mecab_sents2, mecab_sents, josa_dic):
    filtered_words = []
    for output in mecab_sents:
        tmp = []
        for token in output:
            if (len(token[0]) > 1) and (token[1][0] == 'N' ) and token[0] not in josa_dic:
                    filtered_words.append(token[0])
                
    counts = Counter(filtered_words)
    vocab = sorted(counts, key=counts.get, reverse=True)
    word2idx = {word.encode("utf8").decode("utf8"): ii for ii, word in enumerate(vocab,1)}
    idx2word = {ii: word for ii, word in enumerate(vocab)}
#     V = len(word2idx)
#     N = total_data
    
    tfidf = TfidfVectorizer(max_features = 30, max_df=0.75, min_df=len(mecab_sents) // 100 + 1)
    
    #generate tf-idf term-document matrix
    A_tfidf_sp = tfidf.fit_transform(mecab_sents2) 
    tfidf_dict = tfidf.get_feature_names()
    data_array = A_tfidf_sp.toarray()
    data = pd.DataFrame(data_array, columns=tfidf_dict)
    max_value = A_tfidf_sp.max(axis = 0).toarray().ravel()
    
    tfidf_dict = dict()
    for key, val in zip(data.columns, max_value):
        tfidf_dict[key] = val
    tfidf_dict = sorted(tfidf_dict.items(), key = lambda t : t[1], reverse = True)
    
    return tfidf_dict

In [6]:
## soy keyword

from soykeyword.proportion import CorpusbasedKeywordExtractor
def soy_keywords(mecab_sents2, name):
    corpusbased_extractor = CorpusbasedKeywordExtractor(
        min_tf=1,
        min_df=2,
        tokenize=lambda x:x.strip().split(),
        verbose=True
    )
    corpusbased_extractor.train(mecab_sents2)
    
    keywords_base = corpusbased_extractor.extract_from_word(
    name,
    min_score=0.7,
    min_frequency= 3
    )

    soyextract_dict = dict()
    for word in keywords_base:
        if word[1] * word[2] > 3:
            soyextract_dict[word[0]] = word[1] * word[2]
    soyextract_dict = sorted(soyextract_dict.items(), key = lambda k:  k[1], reverse = True)
    return soyextract_dict

In [50]:
## total
def outputs(mecab_sents2, name, mecab_sents, josa_dic, tags):
    
    textrank_keywords = textrank(mecab_sents2, name)
    tfidf_dict = word2vec_tfidf(mecab_sents2, mecab_sents, josa_dic)
    soyextract_dict = soy_keywords(mecab_sents2, name)  
    
    # 각 extractor score 점수를 정규화 시키기 위해 score 가져오기
    li1 = [val[1] for val in textrank_keywords]
    li2 = [val[1] for val in tfidf_dict]
    li3 = [val[1] for val in soyextract_dict]
    
    # 3개 extractor에서 나온 값들 정규화함(result2의 경우, tf-idf가 더 중요한 피쳐로 가중치 줌)
    result1 = [(key, (val - np.mean(li1)) / np.std(li1) ) for key, val in textrank_keywords]
    result2 = [(key, (val - (np.mean(li2) * (2/3))) / np.std(li2) ) for key, val in tfidf_dict]
    result3 = [(key, (val - np.mean(li3)) / np.std(li3) ) for key, val in soyextract_dict]

    # 음식사전에 있으면 추가
    with open('./data/restaurants/food_dic.txt', 'r', encoding = 'utf-8') as f:
        s = f.readlines()
    foods = [word.replace('\n', '') for word in s]
    # 목적사전에 있으면 추가
    withwho = [  '남친', '남자친구', '여친', '여자친구', '부모님', '어른', '회사동기','동기','친구', '데이트', '회식',
               '상견례', '연말모임', '기념일', '생일', '생신', '소개팅']
     
    # 합침
    tmp_dic = dict()
    for word in result1:
        if word in foods :
            tmp_dic[word[0]] = word[1] * 5
        elif word in withwho:
            tmp_dic[word[0]] = word[1] * 3
        else:
            tmp_dic[word[0]] = word[1]

    for word in result2:
        if word[0] in tmp_dic.keys() and word[0] in foods :
            tmp_dic[word[0]] += word[1] * 7
        elif word[0] in tmp_dic.keys() and word[0] in withwho:
            tmp_dic[word[0]] += word[1] * 5
        elif word[0] in tmp_dic.keys():
            tmp_dic[word[0]] += word[1]
            
        elif (word[0] in foods  ):
            tmp_dic[word[0]] = word[1] * 5
        elif word[0] in withwho:
            tmp_dic[word[0]] = word[1] * 3
        else:
            tmp_dic[word[0]] = word[1]

    for word in result3:
        if word[0] in tmp_dic.keys() and word[0] in foods :
            tmp_dic[word[0]] += word[1] * 7
        elif word[0] in tmp_dic.keys() and word[0] in withwho:
            tmp_dic[word[0]] += word[1] * 5
        elif word[0] in tmp_dic.keys():
            tmp_dic[word[0]] += word[1]
            
        elif (word[0] in foods  ):
            tmp_dic[word[0]] = word[1] * 5
        elif word[0] in withwho:
            tmp_dic[word[0]] = word[1] * 3
        else:
            tmp_dic[word[0]] = word[1]
               
    tmp_dic = dict(sorted(tmp_dic.items(), key = lambda k : k[1], reverse = True))
#     print('sorted values')     
#     print(tmp_dic)
   
    final_result = [key for key, val in tmp_dic.items() if (val > 0.2 and key != name and key not in name) or (key in foods ) or (key in withwho) \
                   or (key in name and key in foods)]
    
    except_dic_2 = ['안녕','여기','요즘', '다가', '한번', '음식점','어요', '부터', '오랜만', '포스팅', '근처', '인근', '음식','후기', '식당', '생각', '어가', '사람', '하나', '구들', '오픈', '출구', 
                    '영업시간', '시작','지난', '처음','하니', '유명', '소개', '방문', '맛있', '날씨', '호점', '브레이크','오늘', '타임', '오후', '오전', '역시', '많은', '얼마', '휴무','이번',
                   '가보', '지점', '찍기', '발견', '인근', '우리', '전문점','선정', '사이', '최고', '이태원', '홍대', '신촌', '한강진역', '이태원역', '신촌역', '홍대역', '경리단길', '까지', '메뉴',
                   '가격', '메뉴판', '가게', '입구', '출구', '맛집', '위치', '서울', '여러분', '추천', '바로', '어서', '장소', '면서']
    
    
    # Mecab으로 -역, -동 같은 지명 이름 제외
    mm = MeCab.Tagger()
    aa = mm.parse(' '.join(final_result)).split('\n')
    for word in aa:
        if ('NNP' in word and '지명+' in word.split('NNP')[-1]) or ('MAG' in word) or ('VV+EC' in word) or ('NNB' in word):
            except_dic_2.append(word.split()[0])
    
    final_result__ = [word for word in final_result if word not in except_dic_2]
  
    print('식당이름: ',name)
    print('네이버 태그: ',tags)
    print('마이 태그 답: ',final_result__, end = ', ')
    

    return name, tags, final_result__


In [51]:
# 폴더 이름
path = './data/restaurants/itaewon/'
files_group = [ word for word in os.listdir(path) if word[-3:] == 'csv']

save_path = './result/itaewon.csv'
final_data = []

# 결과 출력
for yy in range(0, len(files_group)):
    after_preprocess = preprocess(path, yy)
    a, b, c = outputs(after_preprocess[0], after_preprocess[1] , after_preprocess[2],  after_preprocess[3], after_preprocess[4])
    tmp = []
    tmp.append(a)
    tmp.append(b)
    tmp.append(c)
    print()
    print('==============================================================')
    final_data.append(tmp)
    
# 데이터 저장
results = pd.DataFrame(final_data)
results.rename(columns = {0: 'name', 1: 'naver_tag', 2: 'my_tag'}, inplace = True)
results.to_csv(save_path, index = False, encoding = 'euc-kr')

총 50 개 중  1 번 째 파일입니다.
총 206개의 문장이 추출되었습니다.


C:\ProgramData\Anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


training was done 225 terms, 206 docs, memory = 0.405 Gb
식당이름:  BNHR
네이버 태그:  ['심플한', '감각적인', '카페', '인테리어소품', '떡집', '밀크티', '디저트', '핫플레이스', '깔끔한내부', '나들이']
마이 태그 답:  ['밀크티', '친구', '카페', '벤허', 'bnhr', '커피', '사평', '분위기', '동반', '계단', '떡국', '플랫화이트'], 
총 50 개 중  2 번 째 파일입니다.
총 200개의 문장이 추출되었습니다.


C:\ProgramData\Anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


training was done 283 terms, 200 docs, memory = 0.406 Gb
식당이름:  DOTS
네이버 태그:  ['친절하고', '고급진', '이국적', '분위기좋은', '세련된', '누들', '레스토랑', '돈가스', '김치볶음밥', '브런치', '핫플레이스', '재방문', '데이트', '나들이', '퓨전중식']
마이 태그 답:  ['다츠', '닷츠', '예약', '레스토랑', '한남', '점심', '분위기', '아시안', '아시아', '브런치', '카페', '가츠', '인기', '웨이팅', '카츠', '산도', '퀴진'], 
총 50 개 중  3 번 째 파일입니다.
총 202개의 문장이 추출되었습니다.


...

training was done 283 terms, 200 docs, memory = 0.406 Gb
식당이름:  패션5
네이버 태그:  ['고급진', '재미있는', '분위기좋은', '아기자기', '세련된', '타르트', '마카롱', '디저트카페', '케이크', '빵집', '핫플레이스', '데이트', '화이트데이', '특별한날', '나들이']
마이 태그 답:  ['베이커리', '생일', '카페', '데이트', '케이크', '파이브', '건물', '딸기', '케익', '디저트', '주차', '천국', '시간', '젤라또'], 
총 50 개 중  49 번 째 파일입니다.
총 214개의 문장이 추출되었습니다.


C:\ProgramData\Anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


training was done 277 terms, 214 docs, memory = 0.406 Gb
식당이름:  프로스트
네이버 태그:  ['빈티지한', '분위기있는', '이국적', '분위기좋은', '고급스러운', '술집', '라운지바', '펍', '칵테일', '샹그리아', '핫플레이스', '데이트', '요즘대세', '특별한날', '나들이']
마이 태그 답:  ['친구', '분위기', '파티', '핫플레이스', '맥주', '라운지', '할로윈', '술집', '칵테일', '연말', '헌팅', '글램', '크리스마스', '거리', '저녁'], 
총 50 개 중  50 번 째 파일입니다.
총 204개의 문장이 추출되었습니다.


C:\ProgramData\Anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


training was done 262 terms, 204 docs, memory = 0.406 Gb
식당이름:  할랄가이즈
네이버 태그:  ['재미있는', '고급진', '이국적', '친절하고', '깨끗한', '치킨', '길거리음식', '중동음식', '샌드위치', '양고기', '나들이', '신선한', '데이트', '재방문', '핫플레이스']
마이 태그 답:  ['케밥', '플래터', '친구', '뉴욕', '저렴', '점심', '미국', '여행', '푸드', '한국', '트럭'], 
